In [ ]:
# from sklearn.datasets import fetch_openml
 
# load dataset
# titanic = fetch_openml('titanic', version=1, as_frame=True)
# df = titanic['data']
# df['survived'] = titanic['target']

In [1]:
import pandas as pd
# load dataset
df = pd.read_csv('/content/train.csv')

FileNotFoundError: ignored

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
 
from sklearn.datasets import fetch_openml
sns.set()

In [ ]:
df.head(3)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
miss_vals = pd.DataFrame(df.isnull().sum() / len(df) * 100)
miss_vals.plot(kind='bar',
    title='Missing values in percentage',
    ylabel='percentage'
    )
 
plt.show()

In [ ]:
df.Survived.value_counts().plot(kind='bar')
 
plt.xlabel('Survival')
plt.ylabel('# of passengers')
plt.title('Number of passengers based on their survival')
plt.show()

In [ ]:
fig, ax = plt.subplots()
 
ax.hist(df.Age.dropna(), label='Not survived')
ax.hist(df['Age'][df.Survived == '1'].dropna(), label='Survived')
 
plt.xlabel('Survival')
plt.ylabel('Age')
plt.title('Survivals by age')
plt.legend()
plt.show()

In [ ]:
df['Survived'] = df.Survived.astype('int')
 
sns.barplot(
    x='Sex',
    y='Survived',
    data=df
)
 
plt.title('Survival by gender')
plt.show()

In [ ]:
sns.countplot(x='Pclass', data=df)
plt.title('Unique survivers by class')
plt.show()

In [ ]:
sns.barplot(x='Pclass', y='Survived', data=df)
plt.title('Percent survivers by class')
plt.show()

In [ ]:
sns.barplot(x='Embarked', y='Survived', data=df)
plt.title('Percent survivers by port of embarkation')
plt.show()

In [ ]:
#from sklearn.datasets import fetch_openml
 
# load dataset
#titanic = fetch_openml('titanic', version=1, as_frame=True)
#df = titanic['data']
#df['survived'] = titanic['target']
 
# Assign Dependent and Independent variables
X = df.drop('Survived', axis=1)
y = df['Survived']

In [ ]:
from sklearn.datasets import fetch_openml
 
X, y = fetch_openml('titanic', version=1, as_frame=True, return_X_y=True)


In [ ]:
X['family'] = X['sibsp'] + X['parch']
X.loc[X['family'] > 0, 'travelled_alone'] = 0
X.loc[X['family'] == 0, 'travelled_alone'] = 1
X.drop(['family', 'sibsp', 'parch'], axis=1, inplace=True)
sns.countplot(x='travelled_alone', data=X)
plt.title('Number of passengers travelling alone')
plt.show()

In [ ]:
# remove high missing value columns
X.drop(['cabin', 'boat', 'body'], axis=1, inplace=True)
 
# remove less interesting features
X.drop(['name','ticket','home.dest'], axis=1, inplace=True)


In [ ]:
from sklearn.impute import SimpleImputer
 
def get_parameters(df):
    parameters = {}
    for col in df.columns[df.isnull().any()]:
        if df[col].dtype == 'float64' or df[col].dtype == 'int64' or df[col].dtype =='int32':
            strategy = 'mean'
        else:
            strategy = 'most_frequent'
        missing_values = df[col][df[col].isnull()].values[0]
        parameters[col] = {'missing_values':missing_values, 'strategy':strategy}
    return parameters
 
parameters = get_parameters(X)
 
for col, param in parameters.items():
    missing_values = param['missing_values']
    strategy = param['strategy']
    imp = SimpleImputer(missing_values=missing_values, strategy=strategy)
    X[col] = imp.fit_transform(X[[col]])
 
X.isnull().sum()

In [ ]:
# handle categorical data
cat_cols = X.select_dtypes(include=['object','category']).columns
dummies = pd.get_dummies(X[cat_cols], drop_first=True)
X[dummies.columns] = dummies
X.drop(cat_cols, axis=1, inplace=True)
X.head()

In [ ]:
# Scale numeric data
from sklearn.preprocessing import StandardScaler
 
# Select numerical columns
num_cols = X.select_dtypes(include=['int64', 'float64', 'int32']).columns
 
# Apply StandardScaler
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])


In [ ]:
from sklearn.model_selection import train_test_split
 
RAND_STATE = 42
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RAND_STATE)


In [ ]:
cross_val_scores = {}
 
models = [
    'LogisticRegression', 
    'KNeighborsClassifier',
    'SVC',
    'RandomForestClassifier',
    'DecisionTreeClassifier'
    ]
 
empty_dict = {
    'best_score':'',
    'best_params':'',
    'score':''
    }
 
for m in models:
    cross_val_scores[m] = empty_dict
 
cross_val_scores

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
 
params = {
    'C': [0.001, 0.01, 0.1, 1.],
    'penalty': ['l1', 'l2']
}
 
log_reg = LogisticRegression(
    random_state=RAND_STATE, 
    class_weight='balanced',
    solver='liblinear'
    )
 
log_reg_cv = GridSearchCV(
    log_reg, 
    param_grid=params, 
    cv=5,
    scoring='accuracy',
)
 
log_reg_cv.fit(X_train, y_train)
 
cross_val_scores['LogisticRegression']['best_score'] = log_reg_cv.best_score_
cross_val_scores['LogisticRegression']['best_params'] = log_reg_cv.best_params_
cross_val_scores['LogisticRegression']['score'] = log_reg_cv.score(X_test, y_test)


In [ ]:
%%time
import numpy as np 
 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
 
params = {'n_neighbors': np.arange(1, 50)}
 
knn = KNeighborsClassifier()
 
knn_cv = GridSearchCV(
    knn, 
    param_grid=params,
    cv=5,
    scoring='accuracy'
    )
 
knn_cv.fit(X_train, y_train)
 
cross_val_scores['KNeighborsClassifier']['best_score'] = knn_cv.best_score_
cross_val_scores['KNeighborsClassifier']['best_params'] = knn_cv.best_params_
cross_val_scores['KNeighborsClassifier']['score'] = knn_cv.score(X_test, y_test)


In [ ]:
%%time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
 
params = {
    'C': [0.001, 0.01, 0.1, 1.],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
}
 
svc = SVC(
    random_state=RAND_STATE,
    class_weight='balanced',
    probability=True,
)
 
svc_cv = GridSearchCV(
    svc, 
    param_grid=params, 
    cv=5,
    scoring='accuracy',
)
 
svc_cv.fit(X_train, y_train)
 
cross_val_scores['SVC']['best_score'] = svc_cv.best_score_
cross_val_scores['SVC']['best_params'] = svc_cv.best_params_
cross_val_scores['SVC']['score'] = svc_cv.score(X_test, y_test)


In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
 
params = {
    'n_estimators': [5, 10, 15, 20, 25], 
    'max_depth': [3, 5, 7, 9, 11, 13],
}
rand_forest = RandomForestClassifier(
    random_state=RAND_STATE,
    class_weight='balanced',
)
 
rf_cv = GridSearchCV(
    rand_forest, 
    param_grid=params, 
    cv=5,
    scoring='accuracy',
)
 
rf_cv.fit(X_train, y_train)
 
cross_val_scores['RandomForestClassifier']['best_score'] = rf_cv.best_score_
cross_val_scores['RandomForestClassifier']['best_params'] = rf_cv.best_params_
cross_val_scores['RandomForestClassifier']['score'] = rf_cv.score(X_test, y_test)


In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
 
params = {
    'max_depth': [3, 5, 7, 9, 11, 13],
}
 
decision_tree = DecisionTreeClassifier(
    random_state=RAND_STATE,
    class_weight='balanced',
)
 
dt_cv = GridSearchCV(
    decision_tree, 
    param_grid=params, 
    cv=5,
    scoring='accuracy',
)
 
dt_cv.fit(X_train, y_train)
 
cross_val_scores['DecisionTreeClassifier']['best_score'] = dt_cv.best_score_
cross_val_scores['DecisionTreeClassifier']['best_params'] = dt_cv.best_params_
cross_val_scores['DecisionTreeClassifier']['score'] = dt_cv.score(X_test, y_test)

pd.DataFrame(cross_val_scores)

In [ ]:
pd.DataFrame(cross_val_scores).T.sort_values(by='best_score',ascending=False)


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
 
from sklearn.datasets import fetch_openml
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
 
# Set random state for reproducibility
RAND_STATE = 42
 
# load data
X, y = fetch_openml('titanic', version=1, as_frame=True, return_X_y=True)
 
# preprocessing
X['family'] = X['sibsp'] + X['parch']
X.loc[X['family'] > 0, 'travelled_alone'] = 0
X.loc[X['family'] == 0, 'travelled_alone'] = 1
X.drop(['family', 'sibsp', 'parch'], axis=1, inplace=True)
X.drop(['cabin', 'boat', 'body'], axis=1, inplace=True)
X.drop(['name','ticket','home.dest'], axis=1, inplace=True)
 
# handle numeric features
numeric_features = ['age','fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
 
# handle categorical features
categorical_features = ['embarked', 'sex', 'pclass', 'travelled_alone']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', OneHotEncoder(handle_unknown='ignore'))])
 
# Create a transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
 
# Run the classifier
classifier = KNeighborsClassifier()
   
# Split into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RAND_STATE)
 
# Set the Pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
 
# Fit the pipeline
model.fit(X_train, y_train)
 
# Predict
y_pred = model.predict(X_test)
 
# Evaluate
print(f'Model score: {model.score(X_test, y_test)}')
 
# compute the classification report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report, plot_confusion_matrix
# plot confusion matrix
color = 'black'
matrix = plot_confusion_matrix(model, X_test, y_test, cmap=plt.cm.Blues)
matrix.ax_.set_title('Confusion Matrix', color=color)
plt.xlabel('Predicted Label', color=color)
plt.ylabel('True Label', color=color)
plt.gcf().axes[0].tick_params(colors=color)
plt.gcf().axes[1].tick_params(colors=color)
plt.show()